# Summary

This study focuses on honing time series forecasting skills using TensorFlow's DNN capabilities. The initial stages encompassed thorough Exploratory Data Analysis (EDA), inclusive of visualizations, and meticulous data wrangling. Subsequently, I established a basic XGBoost model without hyperparameter tuning, aiming primarily to extract feature importance and set a performance baseline for comparison. Following this, I adapted the data for univariate time series forecasting, utilizing Tensor to create DNN models incorporating LSTM CNN layers. Excitingly, the DNN model exhibited superior performance compared to the XGBoost model in both validation and on the public leaderboard.

As I'm relatively new to this domain, I'm open to feedback and suggestions for improvement. Constructive critique is greatly appreciated, as I'm earnestly learning the ropes. And please rate my notebook if you find it helpful. 

#### Library Importation and Compute Resource Assessment

In [ ]:
!pip install --upgrade scikit-learn

In [ ]:
#Checking GPU to understand how much memory available for training
!nvidia-smi

In [ ]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

In [ ]:
#Enabling memory growth for GPU
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)
import tensorflow as tf
tf.config.list_physical_devices('GPU') 

In [ ]:
import IPython
import IPython.display
import pickle
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from dataclasses import dataclass
import pandas as pd
import numpy as np
import keras_tuner as kt
from tensorflow.keras import regularizers
from tensorflow.keras.layers import LeakyReLU
import gc
from tensorflow.keras.callbacks import Callback

In [ ]:
# for reproducibility
seed0=1337
np.random.seed(seed0) 
tf.keras.utils.set_random_seed(seed0)
tf.config.experimental.enable_op_determinism()
tf.random.set_seed(seed0)

In [ ]:
#to avoid seeing shortened text data in pandas cell
pd.set_option('display.max_colwidth', None)

In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Exploratory Data Analysis, Visualization, and Data Wrangling

In [ ]:
#reading datasets
train=pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/train.csv")
test=pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/test.csv")
holiday_events=pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv")
oil=pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/oil.csv")
stores=pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/stores.csv")
transactions=pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/transactions.csv")

In [ ]:
#checking out dataset content and shape
display("Train",train.head(2),train.tail(2), train.shape, "*"*66)
display("Test",test.head(2),test.tail(2), test.shape, "*"*66)
display("Holiday Events",holiday_events.head(2),holiday_events.tail(2), holiday_events.shape, "*"*66)
display("Oil",oil.head(2),oil.tail(2), oil.shape, "*"*66)
display("Stores",stores.head(2),stores.tail(2), stores.shape, "*"*66)
display("Transactions",transactions.head(2),transactions.tail(1), transactions.shape, "*"*66) 

In [ ]:
#converting date feature to datetime data type
train["date"] = pd.to_datetime(train.date)
test["date"] = pd.to_datetime(test.date)
holiday_events["date"] = pd.to_datetime(holiday_events.date)
oil["date"] = pd.to_datetime(oil.date)
transactions["date"] = pd.to_datetime(transactions.date)

#### Visualising time series data 

In [ ]:
#Generic helper function to plot time variant data
def plot_series(time, series, format="-", start=0, end=None):
    fig, ax = plt.subplots(figsize=(14,5))
    plt.plot(time[start:end], series[start:end], format)
    plt.xlabel("Time")
    plt.ylabel("Sales")
    plt.grid(True)
    plt.show()
    plt.close()

In [ ]:
plot_series(train["date"], train["sales"], format="-", start=0, end=None)

In [ ]:
# Zooming into 2017
plot_series(train["date"], train["sales"], format="-",
            start=-len(train["date"][train["date"] >=pd.to_datetime("2017-01-01")]), end=None)

In [ ]:
# checking trend seasonality and residuals
import statsmodels.api as sm
t_series = train.groupby(['date'])['sales'].agg(['mean']).reset_index().rename(columns={'mean': 'msales'})
t_series = t_series.set_index('date')
sample= t_series['msales'].resample('MS').mean()
decomposition = sm.tsa.seasonal_decompose(sample, model='additive')
fig = decomposition.plot()
fig.set_size_inches((14, 6))
fig.tight_layout()
plt.show()
plt.close()

### Joining holiday-oil-stores with train and test sets 

Note: I won't use transactions data because it isn't available for test set and I don't want to dedicate time to extract features utilising it but it can certainly provide useful information and worth engineering in a deeper study.

In [ ]:
# helper function to do merge datasets 
def joins(df,holiday_events,oil,stores):
    print(df.shape)
    holiday_events=holiday_events.drop_duplicates(subset=['date'], keep='last')
    df_holiday=pd.merge(df,holiday_events,how="left",on='date', validate="many_to_one")
    print(f"Shape after merging with holiday {df_holiday.shape}")
    df_holiday_oil=pd.merge(df_holiday,oil,how="left",on='date')
    print(f"Shape after merging with holiday + oil {df_holiday_oil.shape}")
    df_holiday_oil_stores=pd.merge(df_holiday_oil,stores,how="left",on="store_nbr",suffixes=("holiday","stores"))
    print(f"Shape after merging with holiday + oil + stores {df_holiday_oil_stores.shape}")
    return df_holiday_oil_stores

In [ ]:
#merging datasets
train_merged=joins(train,holiday_events,oil,stores)
display(train_merged.head(1),train_merged.tail(1))
test_merged=joins(test,holiday_events,oil,stores)
display(test_merged.head(1),test_merged.tail(1))

### Extracting month and weekday

In [ ]:
# extracting time features for train dataset
train_merged['day_of_week'] = train_merged['date'].dt.day_of_week
train_merged['day_of_week'] = train_merged['day_of_week']+1
train_merged['month'] = train_merged['date'].dt.month
train_merged['year'] = train_merged['date'].dt.year

In [ ]:
# extracting time features for test dataset
test_merged['day_of_week'] = test_merged['date'].dt.day_of_week
test_merged['day_of_week'] = test_merged['day_of_week']+1
test_merged['month'] = test_merged['date'].dt.month
test_merged['year'] = test_merged['date'].dt.year

In [ ]:
# printing basic properties of features
display(train_merged.describe(include="all").T,train_merged.isna().sum() )
display(test_merged.describe(include="all").T,test_merged.isna().sum())

### Changing transfered holidays to normal days

In [ ]:
#checking value counts of holiday types
display(train_merged["typeholiday"].value_counts(dropna=False))
# checking categories of transferred feature of holiday dataset
display(train_merged["transferred"].unique())

In [ ]:
# helper function to convert transfered holidays to normal days
def transfer_holiday_fix(df):
    df["typeholiday"]=np.where(df["transferred"]==True,'NDay',df["typeholiday"])
    df["typeholiday"]=np.where(df["typeholiday"]=='Work Day','NDay',df["typeholiday"])
    df["typeholiday"]=df["typeholiday"].fillna("NDay")
    return df

In [ ]:
train_merged=transfer_holiday_fix(train_merged)
test_merged=transfer_holiday_fix(test_merged)
# checking new structure of holiday type column
display(train_merged["typeholiday"].value_counts(dropna=False))
display(test_merged["typeholiday"].value_counts(dropna=False))

#### Removing the key columns

In [ ]:
#helper function to remove id columns used for joining datasets, 
# note I made decision not to use store_nbr, because I am not using transactions to extract features
# but it can work as category feature for certain models, 
# my decision was because it is an idendifier and useful information it contains also exist in store type and cluster
def select_relevant(df):
    print(f"shape before removing columns {df.shape}")
    features=["date","family","sales","onpromotion","typeholiday","dcoilwtico","city","state",
              "typestores","cluster","day_of_week","month","year"]
    if "sales" in df.columns:
        df= df[features]
    else:
        features.remove("sales")
        df= df[features]
    print(f"Shape after removing columns {df.shape}")
    return df

In [ ]:
train_merged=select_relevant(train_merged)
test_merged=select_relevant(test_merged)

### Fixing the issue with missing values in oil prices by interpolating nans and 0s at oil price

In [ ]:
# plotting current distribution of oil price vs time
plot_series(train_merged["date"], train_merged["dcoilwtico"], format="-", start=0, end=None)

In [ ]:
# helper function to interpolate oil price
def interpolate_oil(df):
    print(f"Shape before interpolating {df.shape}")
    df["dcoilwtico"]=np.where(df["dcoilwtico"] ==0, np.nan, df["dcoilwtico"])
    df.dcoilwtico.interpolate(limit_direction='both',inplace=True)
    print(f"Shape after interpolating {df.shape}")
    return df

In [ ]:
train_merged=interpolate_oil(train_merged)
test_merged=interpolate_oil(test_merged)
display(f"Number of nan in train dataset oil price after interpolation {train_merged['dcoilwtico'].isna().sum()}")
display(f"Number of nan in test dataset oil price after interpolation {test_merged['dcoilwtico'].isna().sum()}")

In [ ]:
plot_series(train_merged["date"], train_merged["dcoilwtico"], format="-", start=0, end=None)

### Visualising change in signal based on features

In [ ]:
# globally setting sns to darkgrid theme
sns.set_theme(style="darkgrid")

In [ ]:
# helper function to plot variation of sales vs time for each class in a feature
def plot_by_category(df,category):
    fig, ax = plt.subplots(figsize=(14,8))
    
    df_grouped=df.groupby(['date',category]).agg({'sales': 'mean'}).reset_index()
    sns.lineplot(data =df_grouped,  x="date", y="sales", hue=category, ax=ax, legend="brief")
    
    # Shrink current axis by 20%
    box = ax.get_position()
    ax.set_position([box.x0, box.y0, box.width * 0.85, box.height])

    # Put a legend to the right of the current axis
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    plt.show()
    plt.close()

In [ ]:
categoricals=[ 'family', 'typeholiday', 'city', 'state', 'typestores', 'cluster']
for cat in categoricals:
    print(cat)
    plot_by_category(train_merged,cat)

In [ ]:
time_features=["day_of_week","month","year"]
for t_feat in time_features:
    print(t_feat)
    plot_by_category(train_merged,t_feat)
    plot_series(train_merged[t_feat], train_merged["sales"], format="-", start=0, end=None)

State and city have very similiar baseline I will use city as it is more specific

In [ ]:
# helper function to plot numeric features vs time in comparison to sales vs time
def plot_by_numeric(df,num_cols):
    fig, ax = plt.subplots(figsize=(14,8))
    cols_to_plot=["date","sales",num_cols]
    df=df[cols_to_plot]
    df_grouped=df.groupby(['date']).agg({'sales': 'mean', num_cols:'mean'}).reset_index()
    g1=sns.lineplot(data=df_grouped, x ="date", y='sales',legend='full', ax=ax)
    g2=sns.lineplot(data=df_grouped, x ="date", y=num_cols,legend='full', ax=ax)
    g1.set(yscale='log')
    g2.set(yscale='log')
    plt.show()
    plt.close()

In [ ]:
numerics=['onpromotion','dcoilwtico']
for num_col in numerics:
    print(num_col)
    plot_by_numeric(train_merged,num_col)

In [ ]:
#helper function to plot numeric features vs time in comparison to sales vs time zooming in after 2017.01.01
def plot_by_numeric_last_year(df,num_cols):
    fig, ax = plt.subplots(figsize=(14,8))
    cols_to_plot=["date","sales",num_cols]
    df=df[cols_to_plot]
    df=df[train["date"] >=pd.to_datetime("2017-01-01")]
    df_grouped=df.groupby(['date']).agg({'sales': 'mean', num_cols:'mean'}).reset_index()
    g1=sns.lineplot(data=df_grouped, x ="date", y='sales', ax=ax)
    g2=sns.lineplot(data=df_grouped, x ="date", y=num_cols, ax=ax)
    g1.set(yscale='log')
    g2.set(yscale='log')
    plt.show()
    plt.close()

In [ ]:
numerics=['onpromotion','dcoilwtico']
for num_col in numerics:
    print(num_col)
    plot_by_numeric_last_year(train_merged,num_col)

In [ ]:
# helper function to extract the features I potentially use for training a model
def extract_features(df):
    print(f"Shape before extracting {df.shape}")
    useful_features=['date', 'family','onpromotion','typeholiday', 'dcoilwtico','city', 'typestores', 'cluster','day_of_week', 'month','year']
    if "sales"in df.columns:
        df=df[useful_features+["sales"]]
    else:
        df=df[useful_features]
    print(f"Shape after extracting{df.shape}")
    return df

In [ ]:
train_merged=extract_features(train_merged)
test_merged=extract_features(test_merged)

In [ ]:
# converting category columns to str type
category_columns=['family', 'typeholiday', 'city', 'typestores', 'cluster']
for column in category_columns:
    train_merged[column] = train_merged[column].astype('str')
    test_merged[column] = test_merged[column].astype('str')

In [ ]:
# converting extracted time features to float
time_columns=["month","day_of_week","year"]
for column in time_columns:
    train_merged[column] = train_merged[column].astype('float')
    test_merged[column] = test_merged[column].astype('float')

In [ ]:
# checking conditions of training and test datasets
display(train_merged.head(2))
display(train_merged.tail(2))
display(train_merged.describe(include="all").T)

In [ ]:
# saving them in to a csv
train_merged.to_csv('train_merged.csv', index=False)
test_merged.to_csv('test_merged.csv', index=False)

#### Common libs & Methods for ML 

In [ ]:
#main modules for preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, Normalizer, MinMaxScaler, PolynomialFeatures
from sklearn.preprocessing import FunctionTransformer, KBinsDiscretizer
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.preprocessing import TargetEncoder

#main modules for designing ML pipeliens
from mlxtend.feature_selection import ColumnSelector
from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer 
from mlxtend.feature_selection import ExhaustiveFeatureSelector as EFS

#main module for evaluation
from sklearn.metrics import mean_squared_log_error, mean_absolute_error, mean_squared_error,r2_score

In [ ]:
# helper function to calculate regression metrics
def calcMetrics(testActualVal, predictions):
    #regression evaluation measures
    data={"RMSLE":[mean_squared_log_error(testActualVal, predictions)**0.5],
         "MAE":[mean_absolute_error(testActualVal, predictions)],
         "RMSE":[mean_squared_error(testActualVal, predictions)**0.5],
         "R2":[r2_score(testActualVal, predictions)]}
    metric_df=pd.DataFrame(data)
    return metric_df

# XGBoost

### Preprocessing for XGBoost

In [ ]:
# helper function to split datset in to train and validation, validation set involves samples after 01.01.2017
def split(df):
    useful_features=['date', 'family','typeholiday','onpromotion', 'dcoilwtico','city', 'typestores',
                     'cluster','day_of_week', 'month','year']
    val_start_index=df.shape[0]-len(df["date"][df["date"] >=pd.to_datetime("2017-01-01")])
    train = df[:val_start_index]
    val=df[val_start_index:]
    X_train = train[useful_features]
    y_train = train["sales"]
    X_valid = val[useful_features]
    y_valid = val["sales"]
    return X_train, y_train, X_valid, y_valid 

In [ ]:
X_train, y_train, X_valid, y_valid =split(train_merged)

In [ ]:
display(X_train.head(1),X_train.tail(1),X_train.shape )
display(X_valid.head(1),X_valid.tail(1), X_valid.shape)

In [ ]:
# A useful relevant referance: https://scikit-learn.org/stable/auto_examples/applications/plot_cyclical_feature_engineering.html
#from sklearn.kernel_approximation import Nystroem
#from sklearn.preprocessing import SplineTransformer

#pipeline for target encoding category features
category_feat=Pipeline(steps=[("target_encode",TargetEncoder(target_type="continuous"))])

# helper functions to be able to get feature names out of functional transformer 
def f_out_sin(self,input_features):
    return input_features
def f_out_cos(self,input_features):
    return input_features

# functions to transform time features with sine cosine transformation 
def sin_transformer(period):
    return FunctionTransformer(lambda x: np.sin(x / period * 2 * np.pi),feature_names_out=f_out_sin)

def cos_transformer(period):
    return FunctionTransformer(lambda x: np.cos(x / period * 2 * np.pi), feature_names_out=f_out_cos)

#adding polynomial transformation on sine_cosine transformed time features to capture linear interactions between time features
time_feat=make_pipeline(
                        ColumnTransformer([
                            #("cyclic_day_of_week", periodic_spline_transformer(7, n_splines=3), ["day_of_week"]),
                            ("day_of_week_sin", sin_transformer(7), ["day_of_week"]),
                            ("day_of_week_cos", cos_transformer(7), ["day_of_week"]),
                            #("cyclic_month", periodic_spline_transformer(12, n_splines=6), ["month"]),
                            ("month_sin", sin_transformer(12), ["month"]),
                            ("month_cos", cos_transformer(12), ["month"]),
                            ("year_sin", sin_transformer(365), ["year"]),
                            ("year_cos", cos_transformer(365), ["year"]),   
                            ],remainder='drop'),
    #Nystroem(kernel="poly", degree=2,n_jobs=-1, n_components=85, random_state=0),
    PolynomialFeatures(degree=2, interaction_only=True, include_bias=False))
# Note: One of my purpose for building this model is to get to get feature importance rating out of it
# therefore I don't want to increase number of features and build a complex model
# If this model be the focus, it might worth trying cyclic spline transformation on time features and kernel approximation to capture
# non linear interactions between time features, this would increase number of features and extend training time and
# increase systems memory requirement for training but as demonstrated in referance from documentation I linked above;
# that approach can be more expressive, the code I commented out is for that approach 

In [ ]:
# names of columns to train xgb
col_names_classic_ml=['family', 'typeholiday','onpromotion', 'dcoilwtico', 'city', 'typestores',
           'cluster', 'day_of_week','month','year']

# names of columns after pipeline transformations, 
# note ordering of this list isn't arbitrary.
# I manually adjusted ordering after getting feature names out of pipeline and verfiying ordering 
col_names_classic_ml_transformed=['family', 'typeholiday','city', 'typestores','cluster',
       'day_of_week_sin','day_of_week_cos', 'month_sin', 'month_cos','year_sin', 'year_cos',
       'day_of_week_sin day_of_week_cos','day_of_week_sin month_sin', 'day_of_week_sin month_cos',
       'day_of_week_sin year_sin', 'day_of_week_sin year_cos','day_of_week_cos month_sin',
       'day_of_week_cos month_cos','day_of_week_cos year_sin', 'day_of_week_cos year_cos','month_sin month_cos',
       'month_sin year_sin','month_sin year_cos','month_cos year_sin','month_cos year_cos', 'year_sin year_cos',
       'onpromotion', 'dcoilwtico']

# building the pipeline to perform feature engineering
preprocess_pipe = Pipeline(steps=[
    ('encoder', ColumnTransformer(
                    transformers=[
                        ("category_trans",category_feat,category_columns),
                        ("time_trans",time_feat,["day_of_week","month","year"] ),
                                ],
                                remainder="passthrough", verbose_feature_names_out=True
                            )),
    ('scaler', MinMaxScaler()),
    ("pandarizer2", FunctionTransformer(lambda x: pd.DataFrame(x, columns =  col_names_classic_ml_transformed)))
                            ],verbose = True)

In [ ]:
preprocess_pipe.fit(X_train[col_names_classic_ml],y_train)

Checking out to see if feature names passed to pandarizer is in correct order, because getting correct output names is important to determine feature importances

In [ ]:
display(preprocess_pipe['encoder'].feature_names_in_)
display(preprocess_pipe['encoder'].get_feature_names_out())

In [ ]:
#fit transform data
preprocess_pipe.fit(X_train[col_names_classic_ml],y_train)
X_train=preprocess_pipe.transform(X_train[col_names_classic_ml])
X_valid=preprocess_pipe.transform(X_valid[col_names_classic_ml])

In [ ]:
display(X_train.head(2),X_train.tail(2),X_train.shape )
display(X_valid.head(2),X_valid.tail(2), X_valid.shape)

### Training XGBoost Regressor

In [ ]:
import xgboost as xgb
from xgboost import XGBRegressor
from xgboost import plot_importance, plot_tree

In [ ]:
# call back to avoid overfitting
early_stop = xgb.callback.EarlyStopping(rounds=10,
                                        metric_name='rmse',
                                        maximize=False,
                                       save_best= True,
                                        )

In [ ]:
# training xgboost model
xgboost_v00=XGBRegressor(random_state=seed0,verbosity=0, n_jobs = -1, reg_lambda=0.005, 
                         learning_rate=0.01, device='gpu',
                          n_estimators=5000, objective='reg:squarederror',
                        callbacks=[early_stop])
xgboost_v00.fit(X_train, y_train, eval_set=[(X_valid, y_valid)],verbose=False)

In [ ]:
# ploting feature importances
_ = plot_importance(xgboost_v00,max_num_features = 10)

### Measuring Performance of XGBoost Regressor

In [ ]:
# creating to dataframe, I will use to visualise predictions vs actual
series_compare=pd.concat([y_train,y_valid], axis=0)
df_compare=pd.DataFrame()
df_compare["actual"]=series_compare
df_compare["date"]=train_merged.date

In [ ]:
# predicting validation set
y_pred_xgb = xgboost_v00.predict(X_valid)
# converting negative predictions to 0
y_pred_xgb=np.where(y_pred_xgb<0,0,y_pred_xgb)
# calculating metrics
calcMetrics(y_valid,y_pred_xgb)

In [ ]:
# additing predictions to comparison dataframe
df_compare["xgb_pred"]=np.nan
df_compare.loc[df_compare.index>=df_compare.shape[0]-len(y_pred_xgb),"xgb_pred"]=y_pred_xgb
df_compare=df_compare.set_index("date")

In [ ]:
# visualising predictions
_ = df_compare[['actual','xgb_pred']].plot(figsize=(15, 5))

In [ ]:
#zooming to 2017 afterwards that was predicted
_ = df_compare[['actual','xgb_pred']].tail(len(y_pred_xgb)).plot(figsize=(15, 5))

### Predicting Test Set with XGBoost Regressor

In [ ]:
# preprocessing test set to rdy for inputting to model
X_test=preprocess_pipe.transform(test_merged[col_names_classic_ml])
#predicting testset
y_pred_test_xgb = xgboost_v00.predict(X_test)

In [ ]:
# writing predictions to a csv for submission
output = pd.DataFrame({'id': test.id, 'sales': y_pred_test_xgb})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

In [ ]:
# saving the model into a pickle file 
#filename= 'StoreSales_XGB.pkl'
#with open(filename, 'wb') as handle:
#    pickle.dump(xgboost_v00, handle, pickle.HIGHEST_PROTOCOL)
# removing model to free memory, 
del xgboost_v00

Public score: 0.94056

# DNN Model : Univariate LSTM - CNN hybrit model

* To incorporate impact of category features into lstm-cnn hybrid dnn, I will create a feature by merging 3 most important category feature together
* To train a dnn for each class in this new featrue, first I will discretise target encode new_feature, then discretise it into bins. This will group classes with similar sales figures together and reduce the number of models needed to be trained

### Preprocessing for DNN

In [ ]:
# reading processed datasets
#train_merged=pd.read_csv("train_merged.csv")
#test_merged=pd.read_csv("test_merged.csv")

# converting extracted time features to float
#time_columns=["month","day_of_week","year"]
#for column in time_columns:
#    train_merged[column] = train_merged[column].astype('float')
#    test_merged[column] = test_merged[column].astype('float')
    
# converting category columns to str type
#category_columns=['family', 'typeholiday', 'city', 'typestores', 'cluster']
#for column in category_columns:
#    train_merged[column] = train_merged[column].astype('str')
#    test_merged[column] = test_merged[column].astype('str')

In [ ]:
# making a copy of dataset I prepered before for training
dnn_df=train_merged.copy()
df_dnn_test_check=test_merged.copy()
# removing the 2nd copy to free memory
del train_merged
del test_merged
dnn_df["cat_gen"]=dnn_df["family"]+dnn_df["city"]+dnn_df["cluster"]
display(f'store_nbr nunique: {train["store_nbr"].nunique()}, family nunique: {dnn_df["family"].nunique()}, city nunique: {dnn_df["city"].nunique()}, cluster nunique: {dnn_df["cluster"].nunique()}, cat_gen nunique: {dnn_df["cat_gen"].nunique()}')

In [ ]:
#col_names_dnn=['cat_gen', 'family','typeholiday', 'city','typestores', 'cluster','dcoilwtico', 'date','day_of_week', 'month', 'year']
col_names_dnn=['cat_gen', 'date']
dnn_df_x=dnn_df[col_names_dnn]
dnn_df_y=dnn_df["sales"]

In [ ]:
# sub pipeline to apply said transformation on derived feature
category_feat_dnn=Pipeline(steps=[("target_encode",TargetEncoder(target_type="continuous")),
                                 ("combiner",KBinsDiscretizer(n_bins=130, strategy='kmeans',
                                  subsample=None, random_state=seed0,encode='ordinal'))
                                 ])

In [ ]:
# engineering the said feature in pipeline
preprocess_pipe_dnn = Pipeline(steps=[
    ('encoder', ColumnTransformer(
                    transformers=[
                        ("category_trans",category_feat_dnn,["cat_gen"])
                                ],
                                remainder="passthrough", verbose_feature_names_out=True
                            )),
    ("pandarizer2", FunctionTransformer(lambda x: pd.DataFrame(x, columns =  col_names_dnn)))
                            ],verbose = True)

In [ ]:
# converting date to string to avoid it getting transformed to float
dnn_df_x = dnn_df_x.astype({'date':'string'})
preprocess_pipe_dnn.fit(dnn_df_x,dnn_df_y)

In [ ]:
# checking if the ordering of feature names are correct
display(preprocess_pipe_dnn['encoder'].feature_names_in_)
display(preprocess_pipe_dnn['encoder'].get_feature_names_out())

In [ ]:
dnn_df_x=preprocess_pipe_dnn.transform(dnn_df_x)
display(dnn_df_x.head(2),dnn_df_x.tail(2),dnn_df_x.shape )

In [ ]:
dnn_training_features= ['cat_gen', 'date']
dnn_df_x=dnn_df_x[dnn_training_features]
#concatenating X-y datasets
dnn_df_processed=pd.concat([dnn_df_x,dnn_df_y], axis=1)
# converting date to datetime dtype
dnn_df_processed["date"] = pd.to_datetime(dnn_df_processed.date)
dnn_df_processed["sales"] = dnn_df_processed["sales"].astype(np.float32)
dnn_df_processed.head(2)

In [ ]:
# removing to free memory
del dnn_df_x
del dnn_df_y

In [ ]:
# checking number of samples in each class of derived feature
display(dnn_df_processed["cat_gen"].value_counts() , dnn_df_processed["cat_gen"].nunique())

In [ ]:
# applying the same transformations to test dataset
df_dnn_test_check["cat_gen"]=df_dnn_test_check["family"]+df_dnn_test_check["city"]+df_dnn_test_check["cluster"]
df_dnn_test_check = df_dnn_test_check.astype({'date':'string'})
df_dnn_test_check=preprocess_pipe_dnn.transform(df_dnn_test_check)
df_dnn_test_check=df_dnn_test_check[dnn_training_features]
df_dnn_test_check["date"] = pd.to_datetime(df_dnn_test_check.date)
# checking if train have any class that test don't have, if so I will remove it
display(f"test: {df_dnn_test_check['cat_gen'].nunique()} number of class in the derived category")
display(f"train: {dnn_df_processed['cat_gen'].nunique()} number of class in the derived category")

There is no category bin that is in train but not in test

In [ ]:
# I will use the 95 day from end of train dataset time series for validation 
Test_sample_size=95
SPLIT_DATE=pd.to_datetime("2017-08-15")-pd.to_timedelta((Test_sample_size), unit='d')
Train_sample_size =365*4+1
Training_interval=pd.to_timedelta((Train_sample_size), unit='d')
train_data_begin_date=SPLIT_DATE-Training_interval
display(f"train_data_begin_date: {train_data_begin_date}")
dnn_df_processed=dnn_df_processed.loc[dnn_df_processed["date"]>=train_data_begin_date]
display(dnn_df_processed.head(2))
#creating a copy of processed dataset for ease of access to unscaled features for later comparison, evaluation
dnn_df_processed_unscaled=dnn_df_processed.copy()
grouped_sample_lenght=len(dnn_df_processed_unscaled[dnn_df_processed_unscaled["cat_gen"]==0].groupby("date")[['cat_gen','sales']].agg({'cat_gen':'mean', 'sales':'median'}))
display(f"lenght of dataframes grouped by cat_gen: {grouped_sample_lenght}")

In [ ]:
#defining parameters for training
SPLIT_TIME =grouped_sample_lenght-Test_sample_size
WINDOW_SIZE = 16
BATCH_SIZE = 32
SHUFFLE_BUFFER_SIZE = 580

In [ ]:
# helper function to devide train dataset into sub-datasets for each class in derived category
# and grouping by date by taking the median of sales
def create_datasets(df):
    dnn_data_sets={}
    fitted_scalers={}
    for val in df["cat_gen"].unique():
        df_segment=df[df["cat_gen"]==val]
        df_segment_grouped=df_segment.groupby("date")[['cat_gen','sales']].agg({'cat_gen':'mean', 'sales':'median'})
        df_segment_grouped=df_segment_grouped.drop(columns=['cat_gen'])
        scaler=MinMaxScaler()
        scaler.fit(df_segment_grouped[["sales"]])
        fitted_scalers[val]=scaler
        df_segment_grouped["sales"]=scaler.transform(df_segment_grouped[["sales"]])
        df_segment_grouped["sales"] = df_segment_grouped["sales"].astype(np.float32)
        dnn_data_sets[val]=df_segment_grouped
    return dnn_data_sets , fitted_scalers
dnn_data_sets, fitted_scalers=create_datasets(dnn_df_processed) 

In [ ]:
# printing a sample of sub-datasets shape info
Sample_of_df_dict=np.random.choice([x for x in dnn_data_sets.keys()], 10 ,replace=False)
for key in Sample_of_df_dict:
    display(f"{key} : {dnn_data_sets[key].shape}")

In [ ]:
# verifying number of samples to be used for training and validation
len_test=len(dnn_data_sets[0][dnn_data_sets[0].index <=SPLIT_DATE])
display(f"train size: {len_test}, test size: {dnn_data_sets[0].shape[0]-len_test}")

In [ ]:
# helper function to parse dataframes to extract time and series
def parse_df(df):
    times =np.array([x for x in range(df.shape[0])])
    series=df["sales"].to_numpy()
    return times, series

In [ ]:
# helper function to devide time and series to train test splits according to time-step
def train_val_split(time, series, time_step):
    time_train = time[:time_step]
    series_train = series[:time_step]
    time_valid = time[time_step:]
    series_valid = series[time_step:]
    return time_train, series_train, time_valid, series_valid

In [ ]:
# helper function to create windowed dataset tensors from series by shifting according to window size 
# and create batches from it for training
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    ds = ds.shuffle(shuffle_buffer)
    ds = ds.map(lambda w: (w[:-1], w[-1]))
    ds = ds.batch(batch_size).prefetch(1)
    return ds

In [ ]:
tf.keras.backend.clear_session()
# helper function to create and compile dnn model
def create_model(WINDOW_SIZE):
    drop_out=0
    activation=LeakyReLU(alpha = 0.01)
    regularizer=regularizers.l2(1e-4)
    model = tf.keras.models.Sequential([
              #tf.keras.layers.Lambda(lambda x: tf.expand_dims(x, axis=-1),
              #            input_shape=[WINDOW_SIZE]),
              tf.keras.layers.Conv1D(filters=64, kernel_size=3,
                          strides=1,
                          activation=activation,
                          padding='causal',
                          input_shape=[WINDOW_SIZE, 1]),
              tf.keras.layers.LSTM(64, return_sequences=True, dropout=drop_out),
              tf.keras.layers.LSTM(64,dropout=drop_out),
              tf.keras.layers.Dense(30, activation=activation, kernel_regularizer = regularizer),
              tf.keras.layers.Dense(10, activation=activation, kernel_regularizer = regularizer),
              tf.keras.layers.Dense(1)
        ])
    learning_rate = 1e-3
    model.compile(loss=tf.keras.losses.Huber(),
                  optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  metrics=["mae"])
    return model
# note: conv1d layer return lstm input in appropriate shape for lstm
# without conv1d layer a lambda layer will be needed to reshape input for lstm, code commented is for this lambda layer

In [ ]:
# callback function to avoid any memory leak issue and reset states in tf.keras
class ClearMemory(Callback):
    def on_train_end(self, epoch, logs=None):
        gc.collect()
        tf.keras.backend.clear_session()

In [ ]:
# call back function to avoid overfitting by early stopping and reduce learning rate on plateau to help convergence
def call():
    #early stopping
    es = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', # metrics to monitor
    patience=20, # how many epochs before stop
    verbose=1,
    mode='min', 
    restore_best_weights=True)
    
    # reducing learning rate on plateau
    rp = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5, # halving learning rate on plateau
    patience=3,
    verbose=1,
    mode='min',
    min_lr=1e-8)
    return es,rp

In [ ]:
# generator function to iterate memory efficently
def ited_dict(dict):
    for key,value in dict.items():
        yield key, value

In [ ]:
# helper function to train model for each class in derived category
# calls the functions above in correct sequence and saves the models and training history to dictionaries 
def train_all(dnn_data_sets,WINDOW_SIZE,BATCH_SIZE,SHUFFLE_BUFFER_SIZE, SPLIT_TIME):
    compiled_models_dict={}
    history_dict={}
    total_num_models=len(dnn_data_sets)
    num_model_to_train=total_num_models
    for key,df in ited_dict(dnn_data_sets):
        print(f"Number of models remained to be trained {num_model_to_train} out of {total_num_models} remained")
        print(f"Training: {key}")
        times, sales = parse_df(df)
        TIME = np.array(times)
        SERIES = np.array(sales)
        
        time_train, series_train, time_valid, series_valid =train_val_split(TIME, SERIES, time_step=SPLIT_TIME)
        
        train_set = windowed_dataset(series_train, window_size=WINDOW_SIZE, batch_size=BATCH_SIZE, 
                                     shuffle_buffer=SHUFFLE_BUFFER_SIZE)
        
        valid_set = windowed_dataset(series_valid, window_size=WINDOW_SIZE, batch_size=BATCH_SIZE, 
                             shuffle_buffer=SHUFFLE_BUFFER_SIZE)
        
        model = create_model(WINDOW_SIZE)
        callbacks = call()
        history = model.fit(train_set, epochs=100, verbose=1,validation_data=valid_set, callbacks=[callbacks,ClearMemory()])
        compiled_models_dict[key]=model
        del model
        history_dict[key]=history
        del history
        num_model_to_train-=1
        IPython.display.clear_output()
        gc.collect()
        tf.keras.backend.clear_session()
    print(f"Trainings finnished: {len(compiled_models_dict)} models trained.")        
    return compiled_models_dict, history_dict

In [ ]:
compiled_models_dict, history_dict=train_all(dnn_data_sets,WINDOW_SIZE,BATCH_SIZE,SHUFFLE_BUFFER_SIZE, SPLIT_TIME)

In [ ]:
# Helper function to plot loss graphs 
def plot_loss_graphs(history, string):
    plt.plot(history.history[string])
    plt.plot(history.history['val_'+string])
    plt.xlabel("Epochs")
    plt.ylabel(string)
    plt.legend([string, 'val_'+string])
    plt.show()
    plt.close()

In [ ]:
# printing 10 randomly selected loss map
Sample_of_loss_graphs=np.random.choice([x for x in history_dict.keys()], 10 ,replace=False,)
for key in Sample_of_loss_graphs:
    print(key)
    plot_loss_graphs(history_dict[key], "loss")

In [ ]:
# removing to free memory
del history_dict

In [ ]:
# helper function for to create windowed dataset without shuffling for fast forecasting 
def model_forecast(model, series, window_size):
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size))
    ds = ds.batch(8).prefetch(1)
    forecast = model.predict(ds)
    return forecast

In [ ]:
#  helper function that calls required helper methods in correct sequence
# and return a forecast for series after split_time (validation data)
def predict_results(models_dict,dnn_data_sets, WINDOW_SIZE,SPLIT_TIME,fitted_scalers):
    predictions={}
    total_num_models=len(dnn_data_sets)
    num_model_to_train=total_num_models
    for key,model in ited_dict(models_dict):
        print(f"Number of datasets remained to be predicted {num_model_to_train} out of {total_num_models} remained")
        print(f"Predicting: {key}")
        times, sales = parse_df(dnn_data_sets[key])
        TIME = np.array(times)
        SERIES = np.array(sales)
        
        dnn_forecast = model_forecast(model,SERIES, WINDOW_SIZE).squeeze()
        dnn_forecast = dnn_forecast[SPLIT_TIME - WINDOW_SIZE:-1]
        
        predictions_df=pd.DataFrame(index=dnn_data_sets[key][SPLIT_TIME :].index)
        predictions_df["predictions"]=dnn_forecast
        del dnn_forecast
        predictions_df["predictions"]=fitted_scalers[key].inverse_transform(predictions_df[["predictions"]])
        predictions_df["predictions"]=np.where(predictions_df["predictions"]<0,0,predictions_df["predictions"])
        predictions[key]=predictions_df
        num_model_to_train-=1
        IPython.display.clear_output()
        gc.collect()
    print(f"Predictions finnished: {len(predictions)} dataset predicted.") 
    return predictions

In [ ]:
predictions_dict=predict_results(compiled_models_dict,dnn_data_sets, WINDOW_SIZE,SPLIT_TIME, fitted_scalers)

In [ ]:
#helper function to plot time series actual vs time series predicted 
def plot_series_multi(time1, series1,series2,label1, label2, format1="--",format2="-", start=0, end=None ):
    fig = plt.figure(figsize=(14,5))
    ax = fig.add_axes([1, 1, 1, 1])
    ax.plot(time1[start:end], series1[start:end])
    ax.plot(time1[start:end], series2[start:end])
    ax.legend(labels = (label1, label2))
    plt.xlabel("Time")
    plt.ylabel("Sales")
    plt.grid(True)
    plt.show()
    plt.close()

In [ ]:
# printing out metrics and actual vs predicted time series graphs for a subsample of predictions for sub-grouped validation datasets
Sample_of_actual_predict_graphs=np.random.choice([x for x in predictions_dict.keys()], 10 ,replace=False,)
for key in Sample_of_actual_predict_graphs:
    #printing out metrics
    print(key)
    display(calcMetrics(predictions_dict[key].to_numpy(),
                        fitted_scalers[key].inverse_transform(dnn_data_sets[key]["sales"][SPLIT_TIME :].to_numpy().reshape(-1, 1))))
    
    time1=dnn_data_sets[key].index.values[SPLIT_TIME :]
    series1=predictions_dict[key].to_numpy()
    series2=fitted_scalers[key].inverse_transform(dnn_data_sets[key]["sales"][SPLIT_TIME :].to_numpy().reshape(-1, 1))
    # plotting the graph
    plot_series_multi(time1, series1,series2,"predictions","actual", format1="--",format2="-", start=0, end=None, )

In [ ]:
# helper function for mapping predictions from sub-grouped datasets to ungrouped full dataset 
def aggragate_predictions(df_valid,predictions_dict):
    df_preds = pd.DataFrame(columns=['predictions'], index=df_valid.index.copy())
    for key,df in ited_dict(predictions_dict):
        for date in df.index:
            df_preds["predictions"]=np.where((df_valid["cat_gen"]==key) & (df_valid["date"]==date),
                                                   df.iloc[df.index.values==date]["predictions"].values[0] ,df_preds["predictions"])
    return df_preds["predictions"].to_numpy()

In [ ]:
# helper function for creating ungrouped full validation dataset and parsing it into X and y
def val_dnn_processed(df):
    val_start_index=df.shape[0]-len(df["date"][df["date"] >SPLIT_DATE])
    val=df[val_start_index:]
    y_valid = val.pop("sales")
    X_valid = val
    return X_valid, y_valid 
X_valid_dnn, y_valid_dnn = val_dnn_processed(dnn_df_processed_unscaled)

In [ ]:
# predicting full-validation datase and calculating metrics
y_pred_dnn=aggragate_predictions(X_valid_dnn,predictions_dict)
calcMetrics(y_pred_dnn,y_valid_dnn)

In [ ]:
# plotting actual vs predictions for validation dataset
plot_series_multi(dnn_df_processed_unscaled.loc[dnn_df_processed_unscaled["date"]>SPLIT_DATE]["date"].values,
                   dnn_df_processed_unscaled.loc[dnn_df_processed_unscaled["date"]>SPLIT_DATE]["sales"].values, y_pred_dnn,
                  "actual","predictions",format1='r^',format2="b^", start=0, end=None )

### Predicting Test Set with Univariate LSTM - CNN hybrit model

In [ ]:
# helper function to devide test into sub-datasets for each class in derived category
def create_grouped_test_datasets(df):
    dnn_data_sets={}
    for val in df["cat_gen"].unique():
        df_segment=df[df["cat_gen"]==val]
        df_segment_grouped=df_segment.groupby("date")[['cat_gen']].agg({'cat_gen':'mean'})
        df_segment_grouped=df_segment_grouped.drop(columns=['cat_gen'])
        dnn_data_sets[val]=df_segment_grouped
    return dnn_data_sets 
dnn_data_set_test =create_grouped_test_datasets(df_dnn_test_check) 

In [ ]:
# helper function that returns predictions for testset for each class in derived category
def predict_results_test(models_dict,dnn_data_sets,dnn_data_set_test, WINDOW_SIZE,fitted_scalers):
    predictions={}
    for key, model in models_dict.items():
        times, sales = parse_df(dnn_data_sets[key])
        SERIES = np.array(sales)
        dnn_forecast = model_forecast(model,SERIES, WINDOW_SIZE).squeeze()
        dnn_forecast = dnn_forecast[-WINDOW_SIZE:]
        predictions_df=pd.DataFrame(index=dnn_data_set_test[key].index)
        predictions_df["predictions"]=dnn_forecast
        predictions_df["predictions"]=fitted_scalers[key].inverse_transform(predictions_df[["predictions"]])
        predictions_df["predictions"]=np.where(predictions_df["predictions"]<0,0,predictions_df["predictions"])
        predictions[key]=predictions_df 
    return predictions
predictions_dict_test=predict_results_test(compiled_models_dict,dnn_data_sets,dnn_data_set_test, WINDOW_SIZE,fitted_scalers)

In [ ]:
# calling aggragate predictions function for mapping predictions from sub-grouped validation datasets to ungrouped full validation dataset
y_pred_dnn_test=aggragate_predictions(df_dnn_test_check,predictions_dict_test)

In [ ]:
# writing predictions to a csv for submission
output = pd.DataFrame({'id': test.id, 'sales': y_pred_dnn_test})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Public score: 0.90424

In [ ]:
# Saving models into pickle files
#for key,model in ited_dict(compiled_models_dict): 
#    docs_add=os.getcwd()+'/dnn_models/'
#    filename= docs_add+key+'.pkl'
#    with open(filename, 'wb') as handle:
#        pickle.dump(model, handle, pickle.HIGHEST_PROTOCOL)
# removing models to free memory 
#del compiled_models_dict

# Concluding Discussion and Future Work:

Several potential enhancements can elevate the quality and scope of this study. One notable concern pertains to the extensive memory consumption of the DNN models' dictionary in my limited local environment. To manage this, I had to reduce the derived category feature's classes to 1/10 of the original count, consequently training fewer DNN models that could fit within my system's memory constraints. Expanding the memory capacity would facilitate training a greater number of models encompassing more classes within the derived category. This approach could enhance model specificity and potentially bolster accuracy.

Furthermore, an avenue for improvement lies in the utilization of multivariate models. While the current model operates as a univariate setup, incorporating the time-varying nature of oil prices as an input feature could notably enhance accuracy. This integration could offer valuable insights by capturing the contextual dynamics of oil price fluctuations within the forecasting framework.

Moreover, the incorporation of engineered transaction features could be a valuable addition to a multivariate model. This inclusion could potentially introduce significant time-varying aspects that may augment the model's predictive capabilities, especially when combined with other relevant variables.

In summary, future endeavors could focus on optimizing memory utilization for more expansive model training, transitioning to multivariate models to incorporate additional impactful features such as oil prices, and exploring the incorporation of transaction-based engineered features to enhance predictive accuracy and robustness. These improvements hold the potential to refine the forecasting models and yield more nuanced and accurate predictions.